In [13]:
#preprocessing for training pipeline
import numpy as np
import pandas as pd
from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from datasets import Dataset
import pickle
from utils.helper import read_py150k_code, read_file_to_string
import regex as re

In [2]:
target_features = [
     'snake_case_var_ratio',
     'snake_case_class_ratio',
     'snake_case_method_ratio',
     'upper_camel_case_var_ratio',
     'upper_camel_case_class_ratio',
     'upper_camel_case_method_ratio',
     'lower_camel_case_var_ratio',
     'lower_camel_case_class_ratio',
     'lower_camel_case_method_ratio',
     'func_decorators_avg',
     'class_decorators_avg',
     'class_parents_avg',
     'comprehensions_avg',
     'generators_avg',
     'lambda_avg',
     'comment_density',
     'ds_density',
]
PY150K_DIR = "data/py150"
PY150K_CODE_DIR = "data/py150/py150_files"
PY150K_TRAIN_AST = "data/py150/python100k_train.json"
PY150K_EVAL_AST = "data/py150/python50k_eval.json"
PY150K_TRAIN_CODE = "data/py150/py150_files/python100k_train.txt"
PY150K_EVAL_CODE = "data/py150/py150_files/python50k_eval.txt"

In [3]:
py150k_df = pd.read_csv("data/py150k_metric_20220524.csv")
bigquery_df = pd.read_csv("data/bigquery_metric_20220526.csv")
bq_content_df = pd.read_csv(f'data/BigQuery/files/cubert_metadata000000000000') 

combined_df = pd.concat([py150k_df, bigquery_df], axis = 0) 
combined_df['file'] = [ x + y  for x, y in zip(combined_df['repository'], combined_df['filepath'])]
bq_content_df['file'] = [ x + y  for x, y in zip(bq_content_df['repository'], bq_content_df['filepath'])]

In [4]:
with open("data/combined_dataset/clusters/feature_set_1/full_feature_clusterer.pickle", "rb") as file:
    cluster_pred = pickle.load(file)
    labels = cluster_pred.labels_
    cluster_num = len(np.unique(labels))
combined_df['labels'] = labels

In [9]:
#get code
code_filenames = read_py150k_code(PY150K_TRAIN_CODE)
#print ((code_filenames[1]))
py150_code = []
for i in range(0, len(code_filenames)):
    #print (i)
    try:
        py150_code.append( read_file_to_string( #regex codefilenames drop data/py150/py150_files/data/ 
            f"{PY150K_CODE_DIR}/{code_filenames[i]}"
        ))
    except: # currently appending empty string for empy files
        print (f"{PY150K_CODE_DIR}/{code_filenames[i]}")
        py150_code.append( "File Error"
        )
ex_files = list(combined_df['file'])
filtered_bq = bq_content_df[bq_content_df['file'].isin(ex_files)]
bigquery_code = list(filtered_bq['content'])

data/py150/py150_files/data/alexandrebarachant/Grasp-and-lift-EEG-challenge/preprocessing/aux.py
data/py150/py150_files/data/pimutils/khal/khal/khalendar/aux.py
data/py150/py150_files/data/ummahusla/Codecademy-Exercise-Answers/Language Skills/Python/Unit 6/Student Becomes the Teacher/Just Average/9-How is everything doing?.py
data/py150/py150_files/data/ummahusla/Codecademy-Exercise-Answers/Language Skills/Python/Unit 7/2-Battleship!/Hit or Miss?/12-Bad Aim.py
data/py150/py150_files/data/pimutils/khal/tests/aux.py
data/py150/py150_files/data/ummahusla/Codecademy-Exercise-Answers/Language Skills/Python/Unit 7/2-Battleship!/Hit or Miss?/10-You Win!.py


In [11]:
#hugging face dataset
print (len(filtered_bq))
combined_code = py150_code + bigquery_code
combined_df['content'] = combined_code
display(combined_df)

115050


,line_count,comment_count,comment_total_len,comment_avg_len,comment_density,id_total,lower_case,id_total_var,lower_case_var,snake_case_ratio,...,repository,filepath,forks,issue_events,stars,parse_error,func_async_count,file,labels,content
0,116.0,2.0,44.0,22.000000,0.017241,1.0,1.0,1.0,1.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,#!/usr/bin/env python\n# -*- coding: utf-8 -*-...
1,363.0,24.0,1244.0,51.833333,0.066116,381.0,214.0,333.0,195.0,0.341207,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,# -*- coding: utf-8 -*-\n# Open Source Initiat...
2,13.0,1.0,21.0,21.000000,0.076923,3.0,3.0,3.0,3.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,"#!/usr/bin/env python\n""""""Django's command lin..."
3,34.0,0.0,0.0,0.000000,0.000000,26.0,18.0,26.0,18.0,0.307692,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,"""""""Installer for hippybot\n""""""\n\nimport os\nc..."
4,11.0,1.0,21.0,21.000000,0.090909,3.0,3.0,3.0,3.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,#!/usr/bin/env python\nimport os\nimport sys\n...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115045,1292.0,201.0,7829.0,38.950249,0.155573,1658.0,1533.0,1646.0,1527.0,0.056695,...,oscarbranson/latools,latools/helpers/plot.py,11.0,77.0,9.0,1.0,NaN,oscarbranson/latoolslatools/helpers/plot.py,-1,"""""""\nPlotting functions.\n\n(c) Oscar Branson ..."
115046,194.0,17.0,633.0,37.235294,0.087629,63.0,42.0,57.0,41.0,0.126984,...,erjac77/ansible-module-f5bigip,library/f5bigip_ltm_monitor_snmp_dca.py,5.0,72.0,6.0,1.0,NaN,erjac77/ansible-module-f5bigiplibrary/f5bigip_...,-1,#!/usr/bin/python\n# -*- coding: utf-8 -*-\n#\...
115047,145.0,1.0,23.0,23.000000,0.006897,111.0,76.0,99.0,76.0,0.270270,...,python-hyper/hyper-h2,test/test_config.py,139.0,376.0,754.0,1.0,NaN,python-hyper/hyper-h2test/test_config.py,-1,"# -*- coding: utf-8 -*-\n""""""\ntest_config\n~~~..."
115048,48.0,0.0,NaN,0.000000,0.000000,53.0,44.0,47.0,44.0,0.094340,...,techbureau/zaifbot,tests/trade/test_trade.py,14.0,198.0,42.0,1.0,NaN,techbureau/zaifbottests/trade/test_trade.py,-1,import unittest\nfrom zaifbot.trade.trade impo...


In [ ]:
#df to dataset
dataset = Dataset.from_pandas(combined_df).train_test_split(test_size=0.2)

